In [11]:
class Category:
    def __init__(self, category):
        # Initialize Category object with a given category name and an empty ledger
        self.category = category
        self.ledger = []

    def deposit(self, amount, description=""):
        # Records deposits in the ledger
        self.ledger.append({"amount": amount, "description": description})

    def withdraw(self, amount, description=""):
        # Records withdrawals in the ledger if there are sufficient funds
        if self.check_funds(amount):
            self.ledger.append({"amount": -amount, "description": description})
            return True
        return False

    def get_balance(self):
        # Calculate and return the current balance
        return sum(item['amount'] for item in self.ledger)

    def transfer(self, amount, budget_category):
        # Transfers funds to another category if there are sufficient funds
        if self.check_funds(amount):
            self.withdraw(amount, f"Transfer to {budget_category.category}")
            budget_category.deposit(amount, f"Transfer from {self.category}")
            return True
        return False

    def check_funds(self, amount):
        # Check if there are sufficient funds for a given amount
        return amount <= self.get_balance()

    def __str__(self):
        # Generate a string representation of the Category object
        output = f"{self.category:*^30}\n"
        for item in self.ledger:
            output += f"{item['description'][:23]:23}{item['amount']:>7.2f}\n"
        output += f"Total: {self.get_balance():.2f}"
        return output


def create_spend_chart(categories):
    # Creates a spending chart based on the list of Category objects
    spendings = [sum(item['amount'] for item in category.ledger if item['amount'] < 0) for category in categories]
    total_spent = sum(spendings)
    percentages = [int((spending / total_spent) * 100) for spending in spendings]

    # Build the chart displaying the percentage spent by category
    chart = "Percentage spent by category\n"
    for i in range(100, -1, -10):
        chart += f"{i:3}| "
        for percentage in percentages:
            chart += "o" if percentage >= i else " "
            chart += "  "
        chart += "\n"

    chart += "    -" + "---" * len(categories) + "\n"
    max_length = max(len(category.category) for category in categories)
    for i in range(max_length):
        chart += "     "
        for category in categories:
            chart += category.category[i] if i < len(category.category) else " "
            chart += "  "
        if i < max_length - 1:
            chart += "\n"

    return chart

In [12]:
# Example usage of the Category and create_spend_chart functions
food = Category("Food")
food.deposit(1000, "initial deposit")
food.withdraw(10.15, "groceries")
food.withdraw(15.89, "restaurant and more food for dessert")
print(food.get_balance())

clothing = Category("Clothing")
food.transfer(50, clothing)
clothing.withdraw(25.55)
clothing.withdraw(100)

auto = Category("Auto")
auto.deposit(1000, "initial deposit")
auto.withdraw(15)

# Print the Category objects and the spending chart
print(food)
print(clothing)
print(create_spend_chart([food, clothing, auto]))

973.96
*************Food*************
initial deposit        1000.00
groceries               -10.15
restaurant and more foo -15.89
Transfer to Clothing    -50.00
Total: 923.96
***********Clothing***********
Transfer from Food       50.00
                        -25.55
Total: 24.45
Percentage spent by category
100|          
 90|          
 80|          
 70|          
 60| o        
 50| o        
 40| o        
 30| o        
 20| o  o     
 10| o  o  o  
  0| o  o  o  
    ----------
     F  C  A  
     o  l  u  
     o  o  t  
     d  t  o  
        h     
        i     
        n     
        g     
